In [3]:
import os
import re

def get_parameters(parameters):
    parameters = parameters.split(',')
    parameters = [parameter.strip() for parameter in parameters]
    result = {}
    # pattern = r"import\s+[\'\"](.+?)[\'\"]\;"
    # pattern_1 = r"\s*(address|string|bool)\s*"
    # pattern_2 = r"\s*(byte|uint)\s*"
    
    for parameter in parameters:
        parameter = parameter.split(' ')
        parameter_type = parameter[0]
        parameter_name = parameter[-1]
        if parameter_type not in result:
            result[parameter_type] = []
        result[parameter_type].append(parameter_name)
    return result

def extract_functions(filepath, extracted={}):
    def find_scopes(filename):
        with open(filename, 'r') as f:
            contract_content = f.read()
        
        scopes = {'interface': {}, 'contract': {}, 'library': {}, 'error': {}}
        current_type = None
        current_name = None
        lines = contract_content.split('\n')
        stack = []
        for line in lines:
            # print(line)
            # print(stack)
            interface_match = re.match(r'^\s*interface\s+(\w+)\s*{', line)
            interface_is_match = re.match(r'^\s*interface\s+(\w+)\s+(is).*{', line)
            contract_match = re.match(r'^(abstract)*\s*contract\s+(\w+)\s*{', line)
            contract_is_match = re.match(r'^(abstract)*\s*contract\s+(\w+)\s+(is).*{', line)
            library_match = re.match(r'^\s*library\s+(\w+)\s*{', line)
            error_match = re.match(r'^(error)\s+(\w+)\s*\((.*?)\)\s*', line)
            if error_match:
                # if '{' in line:
                #     stack.append('{')
                current_type = 'error'
                current_name = error_match.group(2)
                scopes[current_type][current_name] = {}
                parameters = error_match.group(3)
                parameters = get_parameters(parameters)
                scopes[current_type][current_name] = parameters
                current_type = None
                current_name = None
                continue

            if interface_is_match:
                # print(contract_is_match)
                if '{' in line:
                    stack.append('{')
                # print(stack)
                current_type = 'contract'
                current_name = interface_is_match.group(1)
                # print(current_name)
                scopes[current_type][current_name] = []
                continue
            if interface_match:
                stack.append('{')
                # print(stack)
                current_type = 'interface'
                current_name = interface_match.group(1)
                scopes[current_type][current_name] = []
                continue
            if contract_is_match:
                # print(contract_is_match)
                if '{' in line:
                    stack.append('{')
                # print(stack)
                current_type = 'contract'
                current_name = contract_is_match.group(2)
                # print(current_name)
                scopes[current_type][current_name] = []
                continue
            if contract_match:
                # print(contract_match)
                stack.append('{')
                # print(stack)
                current_type = 'contract'
                current_name = contract_match.group(2)
                scopes[current_type][current_name] = []
                continue
            if library_match:
                stack.append('{')
                # print(stack)
                current_type = 'library'
                current_name = library_match.group(1)
                scopes[current_type][current_name] = []
                continue
            
            if '{' in line:
            # if re.match(r'*{*', line):
                stack.append('{')
                # print(stack)
                scopes[current_type][current_name].append(line)
                continue
                
            if re.match(r'^\s*}\s*$', line):
                stack.pop()
                # print(stack)
                if len(stack) == 0:
                    current_type = None
                    current_name = None
                else:
                    scopes[current_type][current_name].append(line)
                continue
                
            if current_type and current_name:
                scopes[current_type][current_name].append(line)
        return scopes
    events_functions = find_scopes(filepath)
    filename = os.path.basename(filepath)
    #print(filename)
    # if extracted[filename] is None:
    extracted[filename] = {'interface': {}, 'contract': {}, 'library': {}, 'error': {}}

    # extracted[filename] = {'interface': {}, 'contract': {}, 'library': {}, 'error': {}}
    error_flag = False
    for contract_type, items in events_functions.items():
        # print(contract_type)
        if contract_type == 'error':
            if error_flag:
                continue
            extracted[filename]['error'] = events_functions['error']
            error_flag = True
            continue
        # extracted[filename] = {}
        for contract_name, code in items.items():
            # print(contract_name)
            extracted[filename][contract_type][contract_name] = {}
            stack = []
            # if contract_name == 'error':
            #     continue
            for index in range(len(code)):
                line = code[index]
                #print(line)
                match = re.match(r'^\s*(event|function|modifier|error)\s+(\w+)\s*\((.*?)\)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_type][contract_name]:
                        extracted[filename][contract_type][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = match.group(3)
                    
                    parameters = get_parameters(parameters)
                    
                    extracted[filename][contract_type][contract_name][function_type][function_name] = parameters
                    continue
                match = re.match(r'^\s*(constructor|fallback|receive)\s*\((.*?)\)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_type][contract_name]:
                        extracted[filename][contract_type][contract_name][function_type] = {}
                    function_name = match.group(1)
                    parameters = match.group(2)
                    parameters = get_parameters(parameters)
                    extracted[filename][contract_type][contract_name][function_type][function_name] = parameters
                    continue
                match = re.match(r'^\s*(event|function|modifier|error)\s+(\w+)\s*\(+', line)
                if match:
                    # print(match)
                    stack.append('(')
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_type][contract_name]:
                        extracted[filename][contract_type][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = line
                    index += 1
                    while index < len(code):
                        line = code[index]
                        if ')' in line:
                            stack.pop()
                            parameters += line.strip()
                            if len(stack) == 0:
                                parameters = re.match(r'^\s*(event|function|modifier|error)\s+(\w+)\s*\((.*?)\)\s*', parameters).group(3)
                                parameters = get_parameters(parameters)
                                extracted[filename][contract_type][contract_name][function_type][function_name] = parameters
                                break
                        parameters += line.strip()
                        index += 1
                    continue
                match = re.match(r'^\s*(modifier)\s+(\w+)\s*', line)
                if match:
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_type][contract_name]:
                        extracted[filename][contract_type][contract_name][function_type] = {}
                    function_name = match.group(2)
                    extracted[filename][contract_type][contract_name][function_type][function_name] = {}
                    # while '(' in line:
                        
                    if '(' in line:
                        stack.append('(')
                        parameters = line
                        index += 1
                        while index < len(code):
                            line = code[index]
                            if ')' in line:
                                stack.pop()
                                parameters += line.strip()
                                if len(stack) == 0:
                                    parameters = re.match(r'^\s*(modifier)\s+(\w+)\s*\((.*?)\)\s*', parameters).group(3)
                                    parameters = get_parameters(parameters)
                                    extracted[filename][contract_type][contract_name][function_type][function_name] = parameters
                                    break
                            parameters += line.strip()
                            index += 1
                    else:
                        index += 1
                        while index < len(code) and not ';' in code[index]:
                            if '(' in line:
                                stack.append('(')
                                parameters = line
                                index += 1
                                while index < len(code):
                                    line = code[index]
                                    if ')' in line:
                                        stack.pop()
                                        parameters += line.strip()
                                        if len(stack) == 0:
                                            parameters = re.match(r'^\s*(modifier)\s+(\w+)\s*\((.*?)\)\s*', parameters).group(3)
                                            parameters = get_parameters(parameters)
                                            extracted[filename][contract_type][contract_name][function_type][function_name] = parameters
                                            break
                                    parameters += line.strip()
                                    index += 1
                                break
                            # line += code[index]
                            index += 1
                        extracted[filename][contract_type][contract_name][function_type][function_name] = {'': ['']}
                    continue
                match = re.match(r'^\s*(constructor|fallback|receive)\s+(\w+)\s*\(+', line)
                if match:
                    stack.append('(')
                    function_type = match.group(1)
                    if function_type not in extracted[filename][contract_type][contract_name]:
                        extracted[filename][contract_type][contract_name][function_type] = {}
                    function_name = match.group(2)
                    parameters = line
                    index += 1
                    while index < len(code):
                        line = code[index]
                        if ')' in line:
                            stack.pop()
                            parameters += line.strip()
                            
                            if len(stack) == 0:
                                parameters = re.match(r'^\s*(constructor|fallback|receive)\s*\((.*?)\)\s*', parameters).group(2)
                                parameters = get_parameters(parameters)
                                extracted[filename][contract_type][contract_name][function_type][function_name] = parameters
                                break
                        parameters += line.strip()
                        index += 1
                    continue
                
    
    return extracted

filepath = '/home/lxm/solidity/solidity/newnew/codes/0x8a759925d5005fd8685535e9f13022ce36602bbd/complete_01_01_TURTLES.sol'
extracted = extract_functions(filepath)


In [4]:
print(extracted)

{'complete_01_01_TURTLES.sol': {'interface': {'IERC20': {'function': {'totalSupply': {'': ['']}, 'balanceOf': {'address': ['account']}, 'transfer': {'address': ['recipient'], 'uint256': ['amount']}, 'allowance': {'address': ['owner', 'spender']}, 'approve': {'address': ['spender'], 'uint256': ['amount']}, 'transferFrom': {'address': ['sender', 'recipient'], 'uint256': ['amount']}}, 'event': {'Transfer': {'address': ['from', 'to'], 'uint256': ['value']}, 'Approval': {'address': ['owner', 'spender'], 'uint256': ['value']}}}, 'IUniswapV2Factory': {'function': {'createPair': {'address': ['tokenA', 'tokenB']}}}, 'IUniswapV2Router02': {'function': {'swapExactTokensForETHSupportingFeeOnTransferTokens': {'uint': ['amountIn', 'amountOutMin', 'deadline'], 'address[]': ['path'], 'address': ['to']}, 'factory': {'': ['']}, 'WETH': {'': ['']}, 'addLiquidityETH': {'address': ['token', 'to'], 'uint': ['amountTokenDesired', 'amountTokenMin', 'amountETHMin', 'deadline']}}}}, 'contract': {'Context': {'fu